In [25]:
import pandas as pd
import scipy.stats

In [2]:
aapl = pd.read_csv('apple.csv', index_col="Date")
msft = pd.read_csv('msft.csv', index_col="Date")

In [3]:
aapl.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2015-12-31,26.752501,26.757500,26.205000,26.315001,24.266081,163649200
2016-01-04,25.652500,26.342501,25.500000,26.337500,24.286833,270597600
2016-01-05,26.437500,26.462500,25.602501,25.677500,23.678219,223164000
2016-01-06,25.139999,25.592501,24.967501,25.174999,23.214844,273829600
2016-01-07,24.670000,25.032499,24.107500,24.112499,22.235069,324377600


In [4]:
msft.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2015-12-31,56.040001,56.189999,55.419998,55.480000,50.298279,27334100
2016-01-04,54.320000,54.799999,53.389999,54.799999,49.681782,53778000
2016-01-05,54.930000,55.389999,54.540001,55.049999,49.908432,34079700
2016-01-06,54.320000,54.400002,53.639999,54.049999,49.001839,39518900
2016-01-07,52.700001,53.490002,52.070000,52.169998,47.297417,56564900


In [11]:
aapl['Daily Return'] = aapl['Adj Close'].pct_change(1)
msft['Daily Return'] = msft['Adj Close'].pct_change(1)

In [6]:
aapl.head()

,Open,High,Low,Close,Adj Close,Volume,Daily Returns
Date,,,,,,,
2015-12-31,26.752501,26.757500,26.205000,26.315001,24.266081,163649200,NaN
2016-01-04,25.652500,26.342501,25.500000,26.337500,24.286833,270597600,0.000855
2016-01-05,26.437500,26.462500,25.602501,25.677500,23.678219,223164000,-0.025059
2016-01-06,25.139999,25.592501,24.967501,25.174999,23.214844,273829600,-0.019570
2016-01-07,24.670000,25.032499,24.107500,24.112499,22.235069,324377600,-0.042205


In [7]:
msft.head()

,Open,High,Low,Close,Adj Close,Volume,Daily Returns
Date,,,,,,,
2015-12-31,56.040001,56.189999,55.419998,55.480000,50.298279,27334100,NaN
2016-01-04,54.320000,54.799999,53.389999,54.799999,49.681782,53778000,-0.012257
2016-01-05,54.930000,55.389999,54.540001,55.049999,49.908432,34079700,0.004562
2016-01-06,54.320000,54.400002,53.639999,54.049999,49.001839,39518900,-0.018165
2016-01-07,52.700001,53.490002,52.070000,52.169998,47.297417,56564900,-0.034783


In [8]:
aapl = aapl.dropna()
msft = msft.dropna()

In [12]:
aapl['Daily Return'].std(), msft['Daily Return'].std()

(0.018706576706010603, 0.017016618889198077)

In [26]:
def compute_sharpe_ratio(data, risk_free_rate = 0):
    mean_return = data['Daily Return'].mean()
    std = data['Daily Return'].std()
    sharpe_ratio = (mean_return - risk_free_rate) / std
    return sharpe_ratio

In [27]:
def compute_annual_sharpe_ratio(data, risk_free_rate = 0):
    mean_return = data['Daily Return'].mean()
    std = data['Daily Return'].std()
    sharpe_ratio = (mean_return - risk_free_rate) / std
    return sharpe_ratio * (252 ** 0.5)

In [28]:
sr_aapl = compute_annual_sharpe_ratio(aapl)
sr_msft = compute_annual_sharpe_ratio(msft)

print(f"The yearly sharpe ratio for APPLE is {sr_aapl} and {sr_msft} for MICROSOFT")

The yearly sharpe ratio for APPLE is 1.2294446751024868 and 1.3140353810669951 for MICROSOFT


In [19]:
def compute_sortino_ratio(data, target, risk_free_rate=0):
    mean_return = data['Daily Return'].mean()
    downside = data[data['Daily Return'] < target]['Daily Return']
    std = downside.std()
    sortino_ratio = (mean_return - risk_free_rate) / std
    return sortino_ratio

In [21]:
sor_aapl = compute_sortino_ratio(aapl, target=0)
sor_msft = compute_sortino_ratio(msft, target=0)

print(f"The daily sortino ratio for APPLE is {sor_aapl} and {sor_msft} for MICROSOFT")

The daily sortino ratio for APPLE is 0.1016753817432765 and 0.10743159493614032 for MICROSOFT


In [24]:
annual_sor_aapl = (252 ** 0.5) * sor_aapl
annual_sor_msft = (252 ** 0.5) * sor_msft

print(f"The yearly sortino ratio for APPLE is {annual_sor_aapl} and {annual_sor_msft} for MICROSOFT")

The yearly sortino ratio for APPLE is 1.6140466473015993 and 1.70542369891232 for MICROSOFT


In [29]:
def compute_prob_sharpe_ratio(data, benchmark = 0):
    sr = compute_sharpe_ratio(data, 0)
    skew = scipy.stats.skew(data['Daily Returns'])
    kurtosis = scipy.stats.kurtosis(data['Daily Returns'], fisher=True)
    
    n = len(data)
    std = ( (1 / (n-1)) * (1 + 0.5 * sr**2 - skew * sr + (kurtosis / 4) * sr**2)) ** 0.5

    ratio = (sr - benchmark) / std
    prob_sharpe_ratio = scipy.stats.norm.cdf(ratio)
    return prob_sharpe_ratio

In [30]:
sr_aapl = compute_prob_sharpe_ratio(aapl)
sr_msft = compute_prob_sharpe_ratio(msft)

print(f"The daily probabilistic sharpe ratio for APPLE is {sr_aapl} and {sr_msft} for MICROSOFT")

The daily probabilistic sharpe ratio for APPLE is 0.9980870592247423 and 0.9989846295493375 for MICROSOFT


In [31]:
annual_sr_aapl = sr_aapl * (252 ** 0.5)
annual_sr_msft = sr_msft * (252 ** 0.5)

print(f"The annual probabilistic sharpe ratio for APPLE is {annual_sr_aapl} and {annual_sr_msft} for MICROSOFT")

The annual probabilistic sharpe ratio for APPLE is 15.844140873002782 and 15.858389360181205 for MICROSOFT
